In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from delta import *

import logging

logging.getLogger("py4j").setLevel(logging.DEBUG)


In [2]:
# Create SparkSession
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/05/25 13:15:36 WARN Utils: Your hostname, luiz resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/25 13:15:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/lipez/OneDrive/Documentos/ProjetosPessoais/atividadeEngDados/spark-delta/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/luiz/.ivy2/cache
The jars for the packages stored in: /home/luiz/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-008e391d-c3b8-4c17-9a97-4d7380d0fc87;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 248ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

In [3]:
spark

In [13]:
csv_path = '../data/HLTVCsGoResults.csv'
delta_path = './spark-warehouse/csgo_matches'

In [23]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Schema corrigido
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("match_url", StringType(), True),
    StructField("match_id", IntegerType(), True),
    StructField("team_A", StringType(), True),
    StructField("team_B", StringType(), True),
    StructField("score_tA", IntegerType(), True),
    StructField("score_tB", IntegerType(), True),
    StructField("competition", StringType(), True),
    StructField("type_of_match", StringType(), True)
])


In [24]:
# Leitura do CSV corrigido (ajuste o caminho e separador)
df = spark.read.option("header", True).option("sep", ";").schema(schema).csv(
    csv_path
)

In [25]:
# 5. Salvar como Delta Lake
df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(delta_path)


25/05/25 13:32:57 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///mnt/c/Users/lipez/OneDrive/Documentos/ProjetosPessoais/atividadeEngDados/data/HLTVCsGoResults.csv


In [26]:
# Criar a tabela apontando para esse Delta
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS csgo_matches
    USING DELTA
    LOCATION '{delta_path}'
""")

DataFrame[]

In [27]:
spark.sql("SELECT * FROM csgo_matches").show()


+-----+--------------------+--------+-----------------+--------------+--------+--------+--------------------+-------------+
|   id|           match_url|match_id|           team_A|        team_B|score_tA|score_tB|         competition|type_of_match|
+-----+--------------------+--------+-----------------+--------------+--------+--------+--------------------+-------------+
|25835|https://www.hltv....| 2319564|         Dignitas|        Cloud9|      14|      16|ESL Pro League Se...|          cch|
|25836|https://www.hltv....| 2321666|       Luminosity|    Complexity|      19|      15|ECS Season 5 Nort...|          trn|
|25837|https://www.hltv....| 2321665|              NRG|        Cloud9|       8|      16|ECS Season 5 Nort...|          trn|
|25838|https://www.hltv....| 2319563|               SK|         OpTic|      16|      13|ESL Pro League Se...|          ovp|
|25839|https://www.hltv....| 2321664|           Cloud9|           NRG|      14|      16|ECS Season 5 Nort...|          inf|
|25840|h

In [ ]:
spark.sql("SELECT * FROM csgo_matches where match_id = 2321666").show()

In [32]:
delta_table = DeltaTable.forPath(spark, "./spark-warehouse/csgo_matches")

In [ ]:
from delta.tables import DeltaTable

delta_table.update(
    condition="match_id = 2321666",
    set={"score_tB": "2"}
)

In [31]:
spark.sql("SELECT * FROM csgo_matches where match_id = 2321666").show()

+-----+--------------------+--------+----------+----------+--------+--------+--------------------+-------------+
|   id|           match_url|match_id|    team_A|    team_B|score_tA|score_tB|         competition|type_of_match|
+-----+--------------------+--------+----------+----------+--------+--------+--------------------+-------------+
|25836|https://www.hltv....| 2321666|Luminosity|Complexity|      19|      10|ECS Season 5 Nort...|          trn|
+-----+--------------------+--------+----------+----------+--------+--------+--------------------+-------------+



In [34]:
delta_table.delete("match_id = 2321666")

In [35]:
spark.sql("SELECT * FROM csgo_matches where match_id = 2321666").show()

+---+---------+--------+------+------+--------+--------+-----------+-------------+
| id|match_url|match_id|team_A|team_B|score_tA|score_tB|competition|type_of_match|
+---+---------+--------+------+------+--------+--------+-----------+-------------+
+---+---------+--------+------+------+--------+--------+-----------+-------------+



In [43]:
from pyspark.sql import Row
from pyspark.sql.functions import col

# Criar o novo Row
new_row = [Row(
    match_url="https://www.hltv.org/matches/9999999/test-vs-test",
    match_id=9999999,
    team_A="TeamTestA",
    team_B="TeamTestB",
    score_tA=16,
    score_tB=14,
    competition="Test Cup",
    type_of_match="bo3"
)]

# Criar o DataFrame
new_data = spark.createDataFrame(new_row)

# Fazer o cast de todos os campos para alinhar com o schema da tabela Delta
new_data = new_data.select(
    col("match_url").cast("string"),
    col("match_id").cast("int"),  # <-- corrigido aqui
    col("team_A").cast("string"),
    col("team_B").cast("string"),
    col("score_tA").cast("int"),
    col("score_tB").cast("int"),
    col("competition").cast("string"),
    col("type_of_match").cast("string")
)

# Agora pode inserir sem erro
new_data.write.format("delta").mode("append").save("./spark-warehouse/csgo_matches")


In [44]:
spark.sql("SELECT * FROM csgo_matches where match_id = 9999999").show()

+----+--------------------+--------+---------+---------+--------+--------+-----------+-------------+
|  id|           match_url|match_id|   team_A|   team_B|score_tA|score_tB|competition|type_of_match|
+----+--------------------+--------+---------+---------+--------+--------+-----------+-------------+
|NULL|https://www.hltv....| 9999999|TeamTestA|TeamTestB|      16|      14|   Test Cup|          bo3|
+----+--------------------+--------+---------+---------+--------+--------+-----------+-------------+

